https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py

i = input_gate, j = new_input, f = forget_gate, o = output_gate


```
def __call__(self, inputs, state, scope=None):
    """Long short-term memory cell (LSTM)."""
    with _checked_scope(self, scope or "basic_lstm_cell", reuse=self._reuse):
      # Parameters of gates are concatenated into one multiply for efficiency.
      if self._state_is_tuple:
        c, h = state
      else:
        c, h = array_ops.split(value=state, num_or_size_splits=2, axis=1)
      concat = _linear([inputs, h], 4 * self._num_units, True)

      # i = input_gate, j = new_input, f = forget_gate, o = output_gate
      i, j, f, o = array_ops.split(value=concat, num_or_size_splits=4, axis=1)

      new_c = (c * sigmoid(f + self._forget_bias) + sigmoid(i) *
               self._activation(j))
      new_h = self._activation(new_c) * sigmoid(o)

      if self._state_is_tuple:
        new_state = LSTMStateTuple(new_c, new_h)
      else:
        new_state = array_ops.concat([new_c, new_h], 1)
      return new_h, new_state
```


In [1]:
import numpy as np
import tensorflow as tf

In [2]:
n_nodes = 1
n_steps = 1
data_x1 = np.array([[1], [1], [0], [0]])

data_dim =  data_x1.shape[1]
n_inputs = data_x1.shape[0]

weights_dim = [data_dim + n_nodes, n_nodes]
weights_dim

[2, 1]

In [3]:
inputs = tf.placeholder(tf.float32, [n_steps, n_inputs, data_dim])
inputs_seq = tf.unstack(inputs)
initial_ltm = tf.placeholder(tf.float32, [n_inputs, n_nodes])
initial_stm = tf.placeholder(tf.float32, [n_inputs, n_nodes])

with tf.name_scope('rec_layer'):

    lstm_rnn_cell = tf.contrib.rnn.core_rnn_cell.LSTMCell(
        num_units=n_nodes, forget_bias=0,)

    lstm_rnn_output, state = tf.contrib.rnn.static_rnn(
        cell=lstm_rnn_cell, 
        initial_state=[initial_ltm, initial_stm], 
        inputs=inputs_seq, 
        dtype=tf.float32)

<div style="width:50%">![](imgs/LSTM3-focus-i.png)</div>

In [4]:
Wi = np.ones(weights_dim) * 0
WC = np.ones(weights_dim) * 1
bi = np.ones(n_nodes) * 100
bC = np.ones(n_nodes) * 0

In [5]:
lt_memory0 = np.array([[ 1.], [ 0.], [ 1.], [ 0.]]) * 0.666
st_memory0 = np.zeros([n_inputs, n_nodes])

Wf = np.ones(weights_dim) * 0
Wo = np.ones(weights_dim) * 0

bf = np.ones(n_nodes) * (100)
bo = np.ones(n_nodes) * (100)

## Order in weights matrix:

In [6]:
lstm_weights = np.concatenate([Wi, WC, Wf, Wo], axis=1)
lstm_biases = np.concatenate([bi, bC, bf, bo])
print(lstm_weights, lstm_biases)

[[ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]] [ 100.    0.  100.  100.]


In [7]:
data_x1 = np.array([[1], [1], [0], [0]]) * np.arctanh(.334)

lstm_weights_var = [v for v in tf.trainable_variables() if v.name == 'rnn/lstm_cell/weights:0'][0]
assign_lstm_weights = tf.assign(lstm_weights_var, lstm_weights)

lstm_biases_var = [v for v in tf.trainable_variables() if v.name == 'rnn/lstm_cell/biases:0'][0]
assign_lstm_biases = tf.assign(lstm_biases_var, lstm_biases)

#print(lstm_weights, lstm_biases)

feed_dict = {inputs: np.array([data_x1]), initial_ltm:lt_memory0, initial_stm: st_memory0}

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    sess.run(assign_lstm_weights)
    sess.run(assign_lstm_biases)
    
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    lstm_out, st = sess.run([lstm_rnn_output, state], feed_dict)
    print(st)
    print('out:')
    print(lstm_out)

LSTMStateTuple(c=array([[ 1.        ],
       [ 0.33399996],
       [ 0.66600001],
       [ 0.        ]], dtype=float32), h=array([[ 0.76159418],
       [ 0.32211033],
       [ 0.58234251],
       [ 0.        ]], dtype=float32))
out:
[array([[ 0.76159418],
       [ 0.32211033],
       [ 0.58234251],
       [ 0.        ]], dtype=float32)]
